## Description of the problem

A description of the problem and a discussion of the background.

Our client, Donut Master Inc., is interested in establishing a strong presence in New York City. However, they have a limited number of locations which they can initially establish. In order to have the best possible locations, we will use Foursquare's location data to find the best locations to establish a new Donut Shop. We shall use k-Means Clustering to classify the different neighborghoods into different 5 different categories and suggest optimal locations to build the first shops of a new donut shop franshise.

Our target audience would be C-suite level executives or managers who need to decide where the company should establish a physical presence. It is also directed at people interested in establishing a franchise in a city.

## Data

We shall use the same data from Foursquare as the one generated for the New York notebook. For all the neighborghoods in New York City, we have the following information: The neighborhood name, the lattitude and longitude of the neighborhood, the venue's name, the venues latitude and longitude and the venue category.

It has 10046 entries and 7 categories. In the next cell, we print the head and the tail of our dataset as an example of our data. From the data we can extract the types of venues nearby donut shops and other similar places. In that way we can classify neighborhoods according to the venues they contain. With the classification algorithms we shall give a score to each neighborhood which shows the best places to build a new donut shop.


In [52]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

nyc_venues = pd.read_csv("coursera_db.csv")

Libraries imported.


In [53]:
def haversine_distance(lat1, lon1, lat2, lon2):
   r = 6371
   phi1 = np.radians(lat1)
   phi2 = np.radians(lat2)
   delta_phi = np.radians(lat2 - lat1)
   delta_lambda = np.radians(lon2 - lon1)
   a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
   res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
   return np.round(res, 2)

In [54]:
nyc_venues.head()
y = pd.get_dummies(nyc_venues, columns = ['Venue Category'], prefix="VC")

In [70]:
venue_value = "Donut Shop"
neighborhood_search = "Baychester"
radius = 100
radius = radius/100
nyc_sub = nyc_venues.loc[nyc_venues['Neighborhood'] == neighborhood_search]
y_sub = y.loc[y['Neighborhood'] == neighborhood_search]

n1 = y_sub.shape[0]

if venue_value not in nyc_sub['Venue Category'].values:
    print("There are no " + venue_value + " in " + neighborhood_search + ".")
Y = nyc_sub[['Venue Category']].head(n1)

venue_matrix  = list(list(0 for j in range(428)) for i in range(n1))

for r in range(n1):
    for r1 in range(r+1,n1):
        lat1 = y_sub.iloc[r,5]
        lat2 = y_sub.iloc[r1,5]
        lon1 = y_sub.iloc[r][6]
        lon2 = y_sub.iloc[r1][6]
        h_dist = haversine_distance(lat1, lon1, lat2, lon2)
        if h_dist <= radius:
            for j in range(428):
                venue_matrix[r][j] += y_sub.iloc[r1][j+7]
                venue_matrix[r1][j] += y_sub.iloc[r][j+7]
                
X = pd.DataFrame(venue_matrix)


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, np.ravel(Y), test_size = 0.2)
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 3)
classifier.fit(X, np.ravel(Y))

y_pred = classifier.predict(X)

swap_list = list()
venue_list = np.ravel(Y)

for i in range(n1):
    if y_pred[i] == venue_value and venue_list[i] != venue_value:
        swap_list.append(i)
        
if len(swap_list) == 0:
    print("There are no suitable sites to convert into a " +  venue_value + ".")
else:
    print("We suggest to convert the following venues into a " + venue_value + ".")
    for i in swap_list:
        print(nyc_sub.iloc[i])


We suggest to convert the following venues into a Donut Shop.
Unnamed: 0                               206
Neighborhood                      Baychester
Neighborhood Latitude                40.8669
Neighborhood Longitude              -73.8358
Venue                        Caridad & Louie
Venue Latitude                       40.8658
Venue Longitude                     -73.8377
Venue Category            Spanish Restaurant
Name: 206, dtype: object
Unnamed: 0                        208
Neighborhood               Baychester
Neighborhood Latitude         40.8669
Neighborhood Longitude       -73.8358
Venue                       Pizza Hut
Venue Latitude                40.8686
Venue Longitude              -73.8338
Venue Category            Pizza Place
Name: 208, dtype: object
Unnamed: 0                           210
Neighborhood                  Baychester
Neighborhood Latitude            40.8669
Neighborhood Longitude          -73.8358
Venue                             SUBWAY
Venue Latitude     

KeyError: "None of [Int64Index([0, 2, 4, 5, 7, 8, 9, 10, 13, 15, 18], dtype='int64')] are in the [index]"

In [69]:
y_sub.head()

,Unnamed: 0,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,VC_Accessories Store,VC_Adult Boutique,VC_Afghan Restaurant,VC_African Restaurant,VC_Airport Terminal,VC_American Restaurant,VC_Antique Shop,VC_Arcade,VC_Arepa Restaurant,VC_Argentinian Restaurant,VC_Art Gallery,VC_Art Museum,VC_Arts & Crafts Store,VC_Arts & Entertainment,VC_Asian Restaurant,VC_Athletics & Sports,VC_Auditorium,VC_Australian Restaurant,VC_Austrian Restaurant,VC_Automotive Shop,VC_BBQ Joint,VC_Baby Store,VC_Bagel Shop,VC_Bakery,VC_Bank,VC_Bar,VC_Baseball Field,VC_Baseball Stadium,VC_Basketball Court,VC_Bath House,VC_Beach,VC_Beach Bar,VC_Beer Bar,VC_Beer Garden,VC_Beer Store,VC_Big Box Store,VC_Bike Rental / Bike Share,VC_Bike Shop,VC_Bike Trail,VC_Bistro,VC_Board Shop,VC_Boat or Ferry,VC_Bookstore,VC_Boutique,VC_Bowling Alley,VC_Boxing Gym,VC_Brazilian Restaurant,VC_Breakfast Spot,VC_Brewery,VC_Bridal Shop,VC_Bridge,VC_Bubble Tea Shop,VC_Buffet,VC_Building,VC_Burger Joint,VC_Burrito Place,VC_Bus Line,VC_Bus Station,VC_Bus Stop,VC_Business Service,VC_Butcher,VC_Cafeteria,VC_Café,VC_Cajun / Creole Restaurant,VC_Cambodian Restaurant,VC_Camera Store,VC_Campground,VC_Candy Store,VC_Cantonese Restaurant,VC_Caribbean Restaurant,VC_Caucasian Restaurant,VC_Cha Chaan Teng,VC_Check Cashing Service,VC_Cheese Shop,VC_Chinese Restaurant,VC_Chocolate Shop,VC_Church,VC_Circus,VC_Climbing Gym,VC_Clothing Store,VC_Club House,VC_Cocktail Bar,VC_Coffee Shop,VC_College Academic Building,VC_College Arts Building,VC_College Basketball Court,VC_College Bookstore,VC_College Cafeteria,VC_College Theater,VC_Colombian Restaurant,VC_Comedy Club,VC_Comfort Food Restaurant,VC_Comic Shop,VC_Community Center,VC_Concert Hall,VC_Construction & Landscaping,VC_Convenience Store,VC_Cooking School,VC_Cosmetics Shop,VC_Coworking Space,VC_Creperie,VC_Cuban Restaurant,VC_Cupcake Shop,VC_Cycle Studio,VC_Czech Restaurant,VC_Dance Studio,VC_Daycare,VC_Deli / Bodega,VC_Department Store,VC_Design Studio,VC_Dessert Shop,VC_Dim Sum Restaurant,VC_Diner,VC_Discount Store,VC_Distillery,VC_Dive Bar,VC_Doctor's Office,VC_Dog Run,VC_Donut Shop,VC_Dosa Place,VC_Drugstore,VC_Dry Cleaner,VC_Dumpling Restaurant,VC_Duty-free Shop,VC_Eastern European Restaurant,VC_Egyptian Restaurant,VC_Electronics Store,VC_Empanada Restaurant,VC_English Restaurant,VC_Entertainment Service,VC_Ethiopian Restaurant,VC_Event Space,VC_Exhibit,VC_Eye Doctor,VC_Factory,VC_Falafel Restaurant,VC_Farm,VC_Farmers Market,VC_Fast Food Restaurant,VC_Field,VC_Filipino Restaurant,VC_Financial or Legal Service,VC_Fish & Chips Shop,VC_Fish Market,VC_Flea Market,VC_Flower Shop,VC_Food,VC_Food & Drink Shop,VC_Food Court,VC_Food Stand,VC_Food Truck,VC_Fountain,VC_French Restaurant,VC_Fried Chicken Joint,VC_Frozen Yogurt Shop,VC_Fruit & Vegetable Store,VC_Furniture / Home Store,VC_Gaming Cafe,VC_Garden,VC_Garden Center,VC_Gas Station,VC_Gastropub,VC_Gay Bar,VC_General Entertainment,VC_German Restaurant,VC_Gift Shop,VC_Gluten-free Restaurant,VC_Go Kart Track,VC_Golf Course,VC_Gourmet Shop,VC_Greek Restaurant,VC_Grocery Store,VC_Gym,VC_Gym / Fitness Center,VC_Gym Pool,VC_Gymnastics Gym,VC_Halal Restaurant,VC_Harbor / Marina,VC_Hardware Store,VC_Hawaiian Restaurant,VC_Health & Beauty Service,VC_Health Food Store,VC_Heliport,VC_Herbs & Spices Store,VC_High School,VC_Himalayan Restaurant,VC_Historic Site,VC_History Museum,VC_Hobby Shop,VC_Hockey Field,VC_Home Service,VC_Hookah Bar,VC_Hostel,VC_Hot Dog Joint,VC_Hotel,VC_Hotel Bar,VC_Hotel Pool,VC_Hotpot Restaurant,VC_IT Services,VC_Ice Cream Shop,VC_Indian Restaurant,VC_Indie Movie Theater,VC_Indie Theater,VC_Indonesian Restaurant,VC_Indoor Play Area,VC_Insurance Office,VC_Intersection,VC_Irish Pub,VC_Israeli Restaurant,VC_Italian Restaurant,VC_Japanese Curry Restaurant,VC_Japanese Restaurant,VC_Jazz Club,VC_Jewelry Store,VC_Jewish Restaurant,VC_Juice Bar,VC_Karaoke Bar,VC_Kebab Restaurant,VC_Kids Store,VC_Kitchen Supply Store,VC_Korean Restaurant,VC_Kosher R

In [33]:
nyc_venues['Neighborhood']

0                        Wakefield
1                        Wakefield
2                        Wakefield
3                        Wakefield
4                        Wakefield
5                        Wakefield
6                        Wakefield
7                        Wakefield
8                        Wakefield
9                       Co-op City
10                      Co-op City
11                      Co-op City
12                      Co-op City
13                      Co-op City
14                      Co-op City
15                      Co-op City
16                      Co-op City
17                      Co-op City
18                      Co-op City
19                      Co-op City
20                      Co-op City
21                      Co-op City
22                      Co-op City
23                      Co-op City
24                      Co-op City
25                     Eastchester
26                     Eastchester
27                     Eastchester
28                  

array(['Dessert Shop', 'Pharmacy', 'Pharmacy', 'Donut Shop',
       'Dessert Shop', 'Donut Shop', 'Donut Shop', 'Pharmacy',
       'Dessert Shop'], dtype=object)

array(['Dessert Shop', 'Pharmacy', 'Pharmacy', 'Ice Cream Shop',
       'Gas Station', 'Donut Shop', 'Sandwich Place', 'Deli / Bodega',
       'Laundromat'], dtype=object)

In [14]:
# type your answer here

nyc_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )



NameError: name 'getNearbyVenues' is not defined

<a id='item3'></a>

<a id='item4'></a>

<a id='item5'></a>